In [1]:
import sys
sys.path.append("../input/timm-master-copy/")

In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader,SequentialSampler
from torchvision import transforms
import timm
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import random
from sklearn.model_selection import KFold
from tqdm.auto import tqdm

In [3]:
#hyperparameters
FOLDS = 7
LR = 0.01
BATCH_SIZE = 16
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
test_pth = "../input/petfinder-pawpularity-score/test"

In [4]:
class PetDataset(Dataset):
    def __init__(self, train_df, size = (384, 384), pth = test_pth, device = DEVICE):
        self.train_df = train_df
        self.size = size
        self.pth = pth
        self.device = DEVICE
        self.valid_transform =  transforms.Compose([  transforms.Resize(self.size),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                      std=[0.229, 0.224, 0.225]),])
    def __len__(self,):
        return len(self.train_df)

    def __getitem__(self, idx):
        filename = self.train_df.iloc[idx]["Id"]
        img = Image.open(self.pth+"/"+filename+".jpg")
        meta_data = np.asarray(self.train_df.loc[idx][1:13].to_list())
        meta_data = torch.tensor(meta_data, dtype = torch.float).to(self.device)
        img = self.valid_transform(img).to(self.device)
        return img, meta_data
    
class PetDataset_Embed(Dataset):
    def __init__(self, train_df, size = (384, 384), pth = test_pth, device = DEVICE):
        self.train_df = train_df
        self.pth = pth
        self.size = size
        self.device = DEVICE
        self.valid_transform =  transforms.Compose([  transforms.Resize(self.size),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                      std=[0.229, 0.224, 0.225]),])
    def __len__(self,):
        return len(self.train_df)

    def __getitem__(self, idx):
        filename = self.train_df.iloc[idx]["Id"]
        meta_data = np.asarray(self.train_df.loc[idx][1:13].to_list())
        for i in range(len(meta_data)):
            meta_data[i] = meta_data[i] + 2*i
        img = Image.open(self.pth+"/"+filename+".jpg")
        img = self.valid_transform(img).to(self.device)
        meta_data = torch.tensor(meta_data, dtype = torch.long).to(self.device)
        return img, meta_data

In [5]:
class PetScorer(nn.Module):
    def __init__(self, backbone = "swin_large_patch4_window12_384"):#swin_large_patch4_window12_384#
        super().__init__()
        
        self.backbone = timm.create_model(backbone, pretrained = False)
        self.dropout = nn.Dropout(0.1)
        self.meta_embeddings = nn.Embedding(24, 256)
        self.dense1 = nn.Linear(1000, 256)
        self.dense2 = nn.Linear(268,64)
        self.dense3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        
    def forward(self, X, features):
        meta_vector = self.meta_embeddings(features)
        X = self.backbone(X)
        #print(X.size())
        X = self.dense1(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = torch.cat([X, features], dim=1)
        X = self.dense2(X)
        X = self.relu(X)
        X = self.dropout(X)
        X = self.dense3(X)
        return self.sigmoid(X)

In [6]:
model = PetScorer().to(DEVICE)
df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

In [7]:
manoj = True
fold_dir = "../input/rate-my-pet"

In [8]:
test_dataset = PetDataset_Embed(df_test)
test_dataloader = DataLoader(test_dataset,
                                     shuffle = False,
                                     batch_size = BATCH_SIZE,
                                     sampler=SequentialSampler(list(range(test_dataset.__len__())))
                                     )

for fold_ in range(FOLDS):
    print('#'*25)
    print('### FOLD',fold_+1)
    print('#'*25)
    scores = []
    if manoj:
        model.load_state_dict(torch.load(fold_dir+"/"+"FOLD_{}_best.pth".format(fold_),map_location=torch.device(DEVICE)))
    else:
        model.load_state_dict(torch.load(f"../input/petmodels/FOLD_{fold_}_best.pth",map_location=torch.device(DEVICE)))
    model.eval()
    for i,(img,metadata) in enumerate(test_dataloader):
        with torch.no_grad():
            y_pred = model(img,metadata)*100
        scores.extend(y_pred.squeeze(dim=1).cpu().numpy())
    df_test[f"Score_{fold_}"] = scores
idk_list = []
for fold_ in range(FOLDS):
    idk_list.append("Score_"+str(fold_))
score = df_test[idk_list].mean(axis = 1)#, 

#########################
### FOLD 1
#########################
#########################
### FOLD 2
#########################
#########################
### FOLD 3
#########################
#########################
### FOLD 4
#########################
#########################
### FOLD 5
#########################
#########################
### FOLD 6
#########################
#########################
### FOLD 7
#########################


In [9]:
df_test["Pawpularity"] = score
df_test = df_test[["Id", "Pawpularity"]]
df_test.to_csv("submission.csv", index=False)
df_test.head()

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,39.038449
1,43a2262d7738e3d420d453815151079e,39.193165
2,4e429cead1848a298432a0acad014c9d,38.974056
3,80bc3ccafcc51b66303c2c263aa38486,39.134216
4,8f49844c382931444e68dffbe20228f4,38.989481


In [10]:
# from sklearn.metrics import mean_squared_error
# mean_squared_error(list(df_test["Pawpularity"]),score)